In [3]:
from astropy.io import fits
import numpy as np
import re
from datetime import datetime

In [5]:
smear_model = fits.open("../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051013/st_2456811983_w.fit")[0]
smear_modelh = smear_model.header
smear_modeld = smear_model.data
xs = smear_modelh["NAXIS1"]
ys = smear_modelh["NAXIS2"]
tEXP = smear_modelh["EXP_0"]
date = smear_modelh["UTC_0"]
date = re.split('[-T]',date)
l_year = 2003
l_month = 5
l_day = 9
l_date = datetime(l_year,l_month,l_day)

year = int(date[0])
month = int(date[1])
day = int(date[2])
date = datetime(year,month,day)
days = date - l_date
DAY = days.days

In [20]:
time = tEXP + (12 * 1024)
print(f"tEXP({tEXP}) + tVCT = {time}")

tEXP(0.0164) + tVCT = 12288.0164


In [4]:
def get_bias(DAY):
    result = 0

    B0 = 3.18 * pow(10,2)
    B1 = -4.12 * pow(10,-2)
    B2 = 2.00 * pow(10,-5)

    result = B0 + (B1 * DAY) + (B2 * pow(DAY,2))

    return result

In [21]:
def get_smear(NV, NH, tEXP, Isky, data):
    tVCT = 12 * 1024 * 10 ** -6#μs

    result = np.zeros((NV,NH))
    Ismear = np.zeros((NV,NH))

    for H in range (NH):
        Ismear[:,H] = 0
        for V in range (NV):
            Ismear[:,H] += (tVCT / (tVCT + tEXP)) * ((data[V,H] - Isky)/1024)
        result[:,H] =data[:,H] - Ismear[:,H]
        # print(Ismear,data[:,H] , "result",result[:,H] )

    return result,Ismear
bias = get_bias(DAY)
bias_data = smear_modeld - bias

result,Ismear = get_smear(xs, ys, tEXP, get_bias(DAY), bias_data)

In [22]:
hdu = fits.PrimaryHDU(Ismear)
hdulist = fits.HDUList([hdu])
hdulist.writeto('smear_model10-6.fits',overwrite=True)

In [23]:
hdu = fits.PrimaryHDU(result)
hdulist = fits.HDUList([hdu])
hdulist.writeto('readout10-6.fits',overwrite=True)